<a href="https://colab.research.google.com/github/alyssa-raphaella/COMP8220/blob/master/45799857_project_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
import pandas as pd
from google.colab import files

In [2]:
# mount drive
drive.mount('/content/gdrive', force_remount=True)
root_path = 'gdrive/My Drive/COMP8220-GColab/'

# import directory from gdrive
dataset_directory = '/content/gdrive/My Drive/COMP8220 - Datasets/Project/'

Mounted at /content/gdrive


In [0]:
# import datasets
images_train = np.load(join(dataset_directory, 'train_images.npy'))
labels_train = np.load(join(dataset_directory, 'train_labels.npy')) # [black, blond, brown, gray]

images_val = np.load(join(dataset_directory, 'val_images.npy'))
labels_val = np.load(join(dataset_directory, 'val_labels.npy'))
files_val = np.load(join(dataset_directory, 'val_files.npy'))

images_test_public = np.load(join(dataset_directory, 'test_public_images.npy'))
files_test_public = np.load(join(dataset_directory, 'test_public_files.npy'))

In [0]:
image_size = 48
hair_colours = ['black', 'blond', 'brown', 'gray']

# reshape and scale values in [0,1] interval 
images_train = images_train.reshape([-1, image_size, image_size, 3]) / 255.0
images_val = images_val.reshape([-1, image_size, image_size, 3]) / 255.0
images_test_public = images_test_public.reshape([-1, image_size, image_size, 3]) / 255.0

In [5]:
# transform from 4D to 2D
vol_shape = images_train.shape[1:4]
n_vol =  np.prod(vol_shape)
images_train_2d = images_train.reshape(images_train.shape[0], n_vol)
images_val_2d = images_val.reshape(images_val.shape[0], n_vol)
images_test_public_2d = images_test_public.reshape(images_test_public.shape[0], n_vol)

print(images_train_2d.shape)
print(images_val_2d.shape)
images_test_public_2d.shape

(12270, 6912)
(98479, 6912)


In [0]:
## SVM train
svm_clf = BaggingClassifier(base_estimator = SVC(kernel = 'linear', random_state = 42), n_estimators = 10, max_samples = 1000, random_state = 42)
svm_clf.fit(images_train_2d, labels_train)
#svm = SVC(kernel="linear")
#svm.fit(images_train_pca[0:30000], labels_train[0:30000])

BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='linear',
                                     max_iter=-1, probability=False,
                                     random_state=42, shrinking=True, tol=0.001,
                                     verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1000, n_estimators=10, n_jobs=None,
                  oob_score=False, random_state=42, verbose=0,
                  warm_start=False)

In [29]:
## RF train
rf_clf = BaggingClassifier(base_estimator=RandomForestClassifier(random_state=42), n_estimators=10, max_samples=1000, random_state=42)
rf_clf.fit(images_train_2d, labels_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
     

In [44]:
## SGD train
sgd_clf = BaggingClassifier(base_estimator=SGDClassifier(loss="hinge", penalty="l2", max_iter=1000), n_estimators=10, max_samples=1000, random_state=42)
sgd_clf.fit(images_train_2d, labels_train)

BaggingClassifier(base_estimator=SGDClassifier(alpha=0.0001, average=False,
                                               class_weight=None,
                                               early_stopping=False,
                                               epsilon=0.1, eta0=0.0,
                                               fit_intercept=True,
                                               l1_ratio=0.15,
                                               learning_rate='optimal',
                                               loss='hinge', max_iter=1000,
                                               n_iter_no_change=5, n_jobs=None,
                                               penalty='l2', power_t=0.5,
                                               random_state=None, shuffle=True,
                                               tol=0.001,
                                               validation_fraction=0.1,
                                               verbose=0, warm_start=False),
 

In [0]:
## Voting train
voting_clf = VotingClassifier( estimators=[('svc', svm_clf), ('rf', rf_clf), ('sgd', sgd_clf)], voting = 'soft')
voting_clf.fit(images_train_2d, labels_train)    

In [0]:
## SVM val predict
svm_predictions = svm_clf.predict(images_val_2d)

In [0]:
## RF val predict
rf_predictions = rf_clf.predict(images_val_2d)

In [0]:
sgd_predictions = sgd_clf.predict(images_val_2d)

In [0]:
voting_predictions = voting_clf.predict(images_val_2d)

In [46]:
print(classification_report(labels_val, sgd_predictions, target_names=hair_colours))

              precision    recall  f1-score   support

       black       0.71      0.84      0.77      4007
       blond       0.77      0.84      0.80      2852
       brown       0.74      0.65      0.69      4507
        gray       0.84      0.43      0.57       904

    accuracy                           0.74     12270
   macro avg       0.77      0.69      0.71     12270
weighted avg       0.75      0.74      0.74     12270



In [0]:
## SVM test predict
svm_test_prediction = svm_clf.predict(images_test_public_2d)

In [0]:
## SVM test predict
rf_test_prediction = rf_clf.predict(images_test_public_2d)

In [0]:
## create Dataframe
df = pd.DataFrame(columns = ['ID', 'Prediction'])
df['ID'] = files_test_public
df['Prediction'] = test_prediction

In [0]:
## export test result
df.to_csv('45799857_01.csv',header=True, index=False)
files.download('45799857_01.csv')

In [0]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(images_train_2d)
pca = PCA(n_components = 0.95)
images_train_pca = pca.fit_transform(images_train_2d)

In [11]:
pca = PCA(n_components = 293)
images_val_pca = pca.fit_transform(images_val_2d)

print(images_train_pca.shape)
print(images_val_pca.shape)

(98479, 293)
(12270, 293)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn_cv = KNeighborsClassifier(n_neighbors=1)
knn_cv.fit(voxel_by_time, labels_train)
#train model with cv of 10 
#cv_scores = cross_val_score(knn_cv, voxel_by_time, labels_train, cv=10)
#print("cv_scores mean:{}".format(np.mean(cv_scores)))
#print(labels_train.shape)
#print(voxel_by_time.shape)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [0]:
predictions = knn_cv.predict(images_val_rs[0:2000])

In [20]:
from sklearn.metrics import classification_report

print(classification_report(labels_val[0:2000], predictions,
	target_names=hair_colours))

              precision    recall  f1-score   support

       black       0.67      0.54      0.60       690
       blond       0.59      0.83      0.69       448
       brown       0.54      0.52      0.53       720
        gray       0.40      0.35      0.37       142

    accuracy                           0.59      2000
   macro avg       0.55      0.56      0.55      2000
weighted avg       0.59      0.59      0.58      2000



In [9]:
images_test_public.shape

(11921, 48, 48, 3)

In [14]:
images_val_pca.shape

(12270, 293)